> **DO NOT EDIT IF INSIDE `tsds` folder** 

# Week 8: Machine learning 1

*Thursday, April 5, 2018*

Machine learning is a technique for learning patterns in data that enable computers to make decisions and predictions. It's probably one of the hottest skills to master as a scientist or engineer in research or industry today. This week, we'll get an overview of what machine learning is, what it can be used for and what its limits are. Without worrying too much about what goes on behind the scenes, we will play around with a few classifiers in Python and test model performance using cross validation. The exercises today cover:

* Model fitting
* Model evaluation
* Using a model for something cool
* Building your own decision tree

**Questions**: [Issue on GitHub](https://github.com/abjer/tsds/issues).

## Exercises

We want to predict whether a Marvel character is a hero or a villain from information that we can extract from their markup. This is a large problem that includes some data wrangling, model fitting and a bit of evaluation. Therefore the problem is split into parts.

### Part 0: Feature representation
In it's raw format, the data cannot be given to a machine learning algorithm. What we must do is extract features from the data and put them into a structured format. This is the same as what we did when we looked at a dog (the data) and extracted into a matrix whether it was fluffy, sad looking, etc. (the features). The feature we will extract here is **team alliances**.

We can represent the team alliances of each character as a row in a matrix where each column corresponds to a particular team. That should look something like this (numbers are made up):

<img src="http://ulfaslak.com/computational_analysis_of_big_data/exer_figures/example_boa.png" width="400">

I've cheated a bit done the feature representation for you. We want to move ahead to the Machine Learning part a bit faster, but all the code you need for doing is in this notebook.

In [22]:
import os, re
import numpy as np
import pandas as pd

def get_alliances(char, faction=None):
    """Return list of alliances for Marvel character."""
    
    if faction is None:
        for faction in ["heroes", "ambiguous", "villains"]:
            faction_chars = [c[:-4] for c in os.listdir("../data/marvel_dataset/%s" % faction)]
            if char in faction_chars:
                break
    
    # Load character markup
    with open("../data/marvel_dataset/%s/%s.txt" % (faction, char)) as fp:
        markup = fp.read()

    # Get alliance field
    alliances_field = re.findall(r"alliances[\w\W]+?\|.+=", markup)
    if alliances_field == []:
        return []

    # Extract teams from alliance field
    return [t[2:-1] for t in re.findall(r"\[\[.+?[\]\|#]", alliances_field[0][10:])]

# Get a complete set of teams
teams = set()
for faction in ["heroes", "ambiguous", "villains"]:
    faction_chars = [c[:-4] for c in os.listdir("../data/marvel_dataset/%s" % faction) if c != ".DS_Store"]
    for char in faction_chars:
        teams.update(get_alliances(char))

# Create a mapping between team name and column index
team_index_dict = dict(zip(list(teams), range(len(teams))))

def get_alliance_vect(char):
    """Return a vector or alliances, given the character name."""
    char_teams = get_alliances(char)

    alliance_vector = [0] * len(teams)
    for ct in char_teams:
        team_index = team_index_dict[ct]
        alliance_vector[team_index] = 1
        
    return np.array(alliance_vector)

def get_faction_X_y_names(faction):
    """Produce feature matrix and target array for a faction."""
    X, y, names = [], [], []
    for char in [c[:-4] for c in os.listdir("../data/marvel_dataset/%s" % faction) if c.endswith(".txt")]:
        if sum(get_alliance_vect(char)) == 0:
            continue
            
        X.append(get_alliance_vect(char))
        
        if faction == "villains":
            y.append(0)
        if faction == "heroes":
            y.append(1)
        if faction == "ambiguous":
            y.append(2)

        names.append(char)

    return np.array(X), np.array(y), names

# Produce target matrix and feature array for each faction
X_villains, y_villains, names_villains = get_faction_X_y_names('villains')
X_heroes, y_heroes, names_heroes = get_faction_X_y_names('heroes')
X_ambiguous, y_ambiguous, names_ambiguous = get_faction_X_y_names('ambiguous')

# Combine
X = np.vstack([X_villains, X_heroes, X_ambiguous])
y = np.hstack([y_villains, y_heroes, y_ambiguous])

# Put into a pandas data frame
teams = [t for t, i in sorted(team_index_dict.items(), key=lambda ti: ti[1])]
data = pd.DataFrame(np.hstack([X, y.reshape(-1, 1)]), columns=teams + ['Class'], index=names_heroes+names_villains+names_ambiguous)

Now, the `data` object is BOTH your feature matrix (all columns from `0:-1`) and your target array (last column `-1`).

### Part 1: Model fitting

> **Ex. 8.1.1**: Train a classifier on **all** heroes (0) and villains (1) and test its accuracy.

>* If your team alliance matrix is `X` and your target array is `y` you can do this by instantiating a model like:
>
        from sklearn.tree import DecisionTreeClassifier
        model = DecisionTreeClassifier()
        model.fit(X, y)  # <--- This is the training/fitting/learning step
        
> The `DecisionTreeClassifier` is an implementation of the standard classification tree. You are free to use any other classifier if you want. Popular ones are Naive Bayes, random forests, support vector machines, feed forward neural networks, logistic regression, and the list goes on. With `sklearn`, they are just as easy to employ as the `DecisionTreeClassifier` classifier (just Google it).


>1. Test the accuracy of your model. You can use the `.predict` method on the `model` object to get predictions for a matrix of data points. Report the accuracy of your model on the same data that you trained the model on, alongside the baseline accuracy of a "dumb" model that only guesses for the majority class.

>2. Report the precision, recall and F1 scores, with respect to the minority class (villains). `sklearn` has implementations that you can use if you are short for time. Extra credit for doing it using only basic linear algebra operations with `numpy`, though.

### Part 2: Model evaluation

> **Ex. 8.2.1**: Investigate how well your model generalizes. You may have noticed that the performance seemed a little too good to be true in Ex 8.1.1.
1. Why did you get such a high accuracy in the previous exercise?
2. Split your data into a test and training set of equal size. Train the model only on the training set and report its accuracy and F1 scores (for both classes) on both the training and test sets.
3. Comment on the difference you observe.

> *Hint: Watch out for unbalanced class proportions! You may want to randomly reorder the rows of your datapoints and target labels so your training and test sets have the same amount of heroes and villains.*

> **Ex. 8.2.2**: Test using cross validation. The performance of a classifier is strongly dependent on the amount of data it is trained on. In Ex. 8.2.1 you train it on only half of the data and test it on the other half. If you rerun that code multiple times, with random 50/50 partitions, you are gonna see a lot of uncertainty in performance. Cross validation solves this problem by training on a larger subset of the data and testing on a smaller one, and taking the average performance over K-folds of this process.
1. Implement cross validation over $K=10$ folds. For each fold you must record the training and test accuracies. In the end, visualize the distributions of test- and training accuracy as histograms in the same plot. It's important that you comment on the result.

### Part 3: Predicting heroness

>**Ex. 8.3.1**: Let's put your classifier to use!
* Retrain your model on all of your data.
* Use the model to estimate the probability that each character is a villain (let's call this *villainness*). You can use the `.predict_proba` method on the model to get probability estimates rather than class assignments.
* **Visualize the "heroness" distribution for all ambiguous characters**. Comment on the result.

### Part 4: Decision trees

In this exercise you will implement the decision making algorithm of a decision tree classifier, step by step.

>**Ex. 8.4.1**: Read about [Shannon entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory).
1. What is it? How is it defined mathematically (write out the formula e.g. using LateX formatting)?
2. Write a function that computes the Shannon-entropy of a probability vector. Compute the Shannon entropy of `p=[0.4, 0.6]`.

>**Ex. 8.4.2**: Split your data into two subsets. One where characters are affiliated with "X-men" and one where they are not.
1. What is the entropy of target labels in each subset?
2. What is the weighted average entropy of the split?
3. Write a function that computes the weighted average entropy of a split, given the data/data-subset and team (name or id) on which to split the data.
4. Plot the distribution of split entropy for all features. Comment on the result. My figure looks [like this](http://ulfaslak.com/computational_analysis_of_big_data/exer_figures/example_6.2.2.4.png).

>**Ex. 8.4.3**: Print the maximum entropy path of a decision tree.
>
>1. Implement the following pseudocode and print the output:<br><br>
>Step 1. Find `team` that gives lowest split entropy for `data`. Print `team`.<br>
>Step 2. Split `data` on `team`, to produce `data0` and `data1`. Print the entropy of each, as well as their weighted avg. entropy.<br>
>Step 3. Overwrite the `data` variable with either `data0` or `data1`, depending on which has the highest entropy.<br>
>Step 4. Stop if there are less than 5 datapoints in `data`. Otherwise start over from 1.<br><br>
>My output looks [like this](http://ulfaslak.com/computational_analysis_of_big_data/exer_figures/example_6.2.3.1.png) for the first five splits.<br><br>
>
>2. Comment on decision path your code takes: How splits are there? Do you notice anything interesting about the final splits? Why do we choose to stop splitting before `data` get smaller than 5?
>3. Train a `sklearn.tree.DecisionTreeClassifier` classifier on the dataset. Initiate the classifier with `criterion='entropy'`. What are the most important features of this classifier? How does this line up with the order of the order of splits you just printed (a comment is fine)?